In [1]:
from dotenv import load_dotenv
# 긴대화 다루기
load_dotenv("../../.env")

True

## Summarize messages

In [2]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import SummarizationMiddleware

agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[
        SummarizationMiddleware( # 대화 요약 미들웨어
            model="gpt-4o-mini",
            trigger=("tokens", 100), # 요약할 토큰의 최대 길이
            keep=("messages", 1) # 유지할 메시지
        )
    ],
)

In [3]:
from langchain.messages import HumanMessage, AIMessage
from pprint import pprint

response = agent.invoke(
    {"messages": [
        HumanMessage(content="달의 수도는?"),
        AIMessage(content="달의 수도는 Lunapolis 입니다.."),
        HumanMessage(content="Lunapolis의 날씨는?"),
        AIMessage(content="하늘을 맑고, 기온은 최대 120C 최저 -100C입니다."),
        HumanMessage(content="Lunapolis의 치즈 개발자의 수는?"),
        AIMessage(content="Lunapolis에는 100,000명의 치즈 개발자가 살고 있어요."),
        HumanMessage(content="치즈 개발자 조합이 파업할 것이라고 생각해?"),
        AIMessage(content="네, 그들은 새 대통령 때문에 행복하지 않다고 생각해요."),
        HumanMessage(content="니가 Lunapolis의 새 대통령이라면 치즈 개발자 조합에 대해 어떻게 대응할 거야?"),
        ]},
    {"configurable": {"thread_id": "1"}}
)

pprint(response)

{'messages': [HumanMessage(content="Here is a summary of the conversation to date:\n\n## SESSION INTENT\nThe user is exploring information about Lunapolis, including its weather, population of cheese developers, and potential labor disputes.\n\n## SUMMARY\nThe user asked about the capital of the Moon, which is Lunapolis. They inquired about the weather, which is clear with temperatures ranging from a maximum of 120°C to a minimum of -100°C. The user also wanted to know the number of cheese developers in Lunapolis, which is 100,000. Additionally, the user questioned whether the cheese developers' union might go on strike, and it was conveyed that they are unhappy with the new president.\n\n## ARTIFACTS\nNone\n\n## NEXT STEPS\nContinue to provide information related to Lunapolis, cheese developers, or any other questions the user may have regarding the Moon or its conditions.", additional_kwargs={'lc_source': 'summarization'}, response_metadata={}, id='c21eab68-c1eb-4096-beae-7a2f102f753

In [4]:
print(response["messages"][0].content)

Here is a summary of the conversation to date:

## SESSION INTENT
The user is exploring information about Lunapolis, including its weather, population of cheese developers, and potential labor disputes.

## SUMMARY
The user asked about the capital of the Moon, which is Lunapolis. They inquired about the weather, which is clear with temperatures ranging from a maximum of 120°C to a minimum of -100°C. The user also wanted to know the number of cheese developers in Lunapolis, which is 100,000. Additionally, the user questioned whether the cheese developers' union might go on strike, and it was conveyed that they are unhappy with the new president.

## ARTIFACTS
None

## NEXT STEPS
Continue to provide information related to Lunapolis, cheese developers, or any other questions the user may have regarding the Moon or its conditions.


## Trim/delete messages

In [ ]:
from typing import Any
from langchain.agents import AgentState
from langchain.messages import RemoveMessage
from langgraph.runtime import Runtime
from langchain.agents.middleware import before_agent
from langchain.messages import ToolMessage

# 에이전트 시작전 모든 도구 메시지를 삭제하도록 함
@before_agent
def trim_messages(state: AgentState, runtime: Runtime) -> dict[str, Any] | None: # 상태와 런타임을 인자로 받음에 유의
    """Remove all the tool messages from the state"""
    messages = state["messages"]

    tool_messages = [m for m in messages if isinstance(m, ToolMessage)]
    
    return {"messages": [RemoveMessage(id=m.id) for m in tool_messages]}

In [6]:
agent = create_agent(
    model="gpt-5-nano",
    checkpointer=InMemorySaver(),
    middleware=[trim_messages],
)

In [ ]:
response = agent.invoke(
    {"messages": [
        HumanMessage(content="내 장치가 동작하지 않아 어떻게 해야돼?"),
        ToolMessage(content="blorp-x7 initiating diagnostic ping…", tool_call_id="1"),
        AIMessage(content="장치가 연결되어 있고 전원이 켜져 있나요?"),
        HumanMessage(content="응, 연결되어 있고 켜져 있어."),
        ToolMessage(content="temp=42C voltage=2.9v … greeble complete.", tool_call_id="2"),
        AIMessage(content="장치에 램프나 인디케이터가 표시되나요?"),
        HumanMessage(content="장치의 온도는?")
        ]},
    {"configurable": {"thread_id": "2"}}
)

pprint(response) # 도구 메시지가 제거 되었음을 알 수 있음

{'messages': [HumanMessage(content='내 장치가 동작하지 않아 어떻게 해야돼?', additional_kwargs={}, response_metadata={}, id='91569801-5422-4ff1-b90a-c1e95b077f11'),
              AIMessage(content='장치가 연결되어 있고 전원이 켜져 있나요?', additional_kwargs={}, response_metadata={}, id='a8329ea0-579c-45c4-b8ec-4db868471916', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content='응, 연결되어 있고 켜져 있어.', additional_kwargs={}, response_metadata={}, id='1886d9b8-59ae-4784-9013-42321f81aa4f'),
              AIMessage(content='장치에 램프나 인디케이터가 표시되나요?', additional_kwargs={}, response_metadata={}, id='e62d0a58-80fd-4883-88a0-abeae729c99a', tool_calls=[], invalid_tool_calls=[]),
              HumanMessage(content='장치의 온도는?', additional_kwargs={}, response_metadata={}, id='83ae8ae5-71db-4f31-8ed6-71021c69e174'),
              AIMessage(content='온도를 확인해 보셨나요? 온도 수치에 따라 대처 방법이 달라지니 먼저 현재 수치를 알려주시면 더 정확하게 도와드릴 수 있습니다. 아래는 일반적인 확인 방법과 시나리오별 조치들입니다.\n\n- 온도 확인 방법 (대상에 따라 다릅니다)\n  - Windows PC: CPU 온도는 보통 기본 모니터링 프로그램이

In [8]:
print(response["messages"][-1].content)

온도를 확인해 보셨나요? 온도 수치에 따라 대처 방법이 달라지니 먼저 현재 수치를 알려주시면 더 정확하게 도와드릴 수 있습니다. 아래는 일반적인 확인 방법과 시나리오별 조치들입니다.

- 온도 확인 방법 (대상에 따라 다릅니다)
  - Windows PC: CPU 온도는 보통 기본 모니터링 프로그램이 필요합니다. 예: HWMonitor, HWiNFO, Core Temp 등을 설치해 CPU 각 코어 온도를 확인하세요.
  - macOS: 기본적으로 온도 수치를 바로 보여주지 않으니 iStat Menus, Macs Fan Control 같은 도구를 사용해 확인합니다.
  - Linux: 터미널에서 lm-sensors를 설치하고 sensors 명령으로 CPU 온도 확인 가능.
  - 스마트폰(안드로이드/아이폰): 설정의 배터리 관련 항목이나 외부 앱(CPU-Z, HW monitor 등)을 사용해 온도를 확인하려고 시도해볼 수 있습니다. 다만 모델마다 경로가 다릅니다.
  - 모니터를 사용할 수 없는 경우: 기기에 따라 제조사에서 제공하는 진단 도구나 앱이 있을 수 있습니다.

- 일반적인 정상 범위
  - 대기(아이들) 시: 약 30–50°C
  - 보통 작업 시: 50–70°C
  - 고부하 상황: 70–85°C
  - 아주 심한 부하 또는 냉각 문제 시 90°C 이상으로 올라갈 수 있음
  - 단, 부품 종류나 기기에 따라 다를 수 있으니 제조사 권장 범위를 우선 참고하세요.

- 온도가 너무 높을 때의 대처
  - 환기 개선: 기기 주변이 통풍이 잘 되도록 하며, 공기 흐름을 차단하는 수납이나 담요 등에서 벗기기.
  - 청소: 먼지로 팬이 원활히 돌아가지 않는 경우가 많으니, 외부 흡입구와 배출구를 청소해 주세요(전원 끄고 수행).
  - 냉각 강화: 가능하면 냉각 패드 사용, 차가운 방에서 사용, 기기를 평평하고 단단한 표면에 놓기.
  - 부하 줄이기: 필요하지 않은 프로그램 종료, 백그라운드에서 과도한 작업이 진행 중인지 확인.
  - 재시작/전원 관리